In [6]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
from tqdm import tqdm

In [7]:
# Hyperparameters
player_names = ["Thomas", "Emily", "Benjamin", "Sophia", "Victoria"]
roles = [0,0,0,1,1]

model = "gpt-4-0314" # "gpt-3.5-turbo-16k" 

role_map = {
    0: "peasant",
    1: "werewolf"
}

In [9]:
n_games = 7

for game in tqdm(range(5, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Init all the players
    players : Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=["The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n"] if name.lower() in werewolf_names else [])
        players[name.lower()].init_player(players=player_names)

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    # Debate stage
    while rounds < 15 and not stop:
        rounds += 1
        next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-4-0314").lower()

        if next_to_speak != "vote":    
            censored, uncensored = players[next_to_speak].get_player_text()
            
            for name in player_names:
                if name.lower() != next_to_speak:
                    players[name.lower()].add_other_text(censored)
        
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            print(uncensored)
        else:
            break

    # Vote stage
    votes = {name.lower(): 0 for name in player_names}

    vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
    conversation_history.append(vote_prompt.content)

    print(vote_prompt.content)

    for name in player_names:
        players[name.lower()].add_other_text(vote_prompt.content)

    for voting_name in player_names:
        if name.lower() in werewolf_names:
            players[name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolfs are (including you): {', '.join(werewolf_names)}. If a wolf die, then you also loose!]\n"
       
        censored, uncensored = players[voting_name.lower()].get_player_text()

        for name in player_names:
            if name.lower() != voting_name:
                players[name.lower()].add_other_text(censored)
        
        conversation_history.append(censored)
        conversation_history_uncensored.append(uncensored)

        voted_for = ""
        search = censored.lower()
        for name in player_names:
            if name.lower() in search:
                voted_for = name.lower()
                search = search.split(name.lower())[-1]
    
        try:
            votes[voted_for] += 1
        except:
            pass

        print(uncensored)
    print(votes)

    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "player_names": player_names,
        "model": model
    }, open(f"../data/experiment_gpt-4-0314_v3/game_number_{game}.json", "w+"), indent=4)

  0%|          | 0/2 [00:00<?, ?it/s]

Thomas: [First, I need to express shock and sadness over Bob's death. Then, I should cast suspicion on someone else, preferably not Benjamin, my fellow werewolf.]

Oh, no! Poor Bob... This is terrible. We need to find out who did this! I have a feeling that Sophia might be hiding something. She was awfully quiet last night. What do you think, Sophia?
Sophia: [Thomas is accusing me of being quiet last night, but I don't think he's necessarily the Werewolf for suggesting that. I should deflect the attention back to him and put him on the defensive, but I also need to keep an eye on the others.]

Oh, really, Thomas? It's interesting that you would single me out so quickly. What do you have to hide? After all, you've just drawn attention to yourself by being the first to accuse someone.
Thomas: [She has a point, I need to be cautious but stick to my aggressive strategy. I should deflect her suspicion by giving a reason for my doubts about her.]

I didn't mean any offense, Sophia. It's just

 50%|█████     | 1/2 [02:36<02:36, 156.59s/it]

Victoria: [My vote is crucial to the outcome. Thomas's sudden change of heart is suspicious, but Benjamin's attempts to shift focus could also be a strategy. I'll cast my vote based on who I think is more likely to be the Werewolf, but I'll keep my aggressive tone.]

I've been listening carefully to all of your words and actions, and I must ultimately trust my instincts. Thomas, your sudden change of heart is indeed suspicious, but Benjamin's consistent attempts to shift the focus from himself strike me as a clever way to avoid suspicion. Thus, I cast my vote for Benjamin.
{'thomas': 2, 'emily': 0, 'benjamin': 3, 'sophia': 0, 'victoria': 0}
Thomas: [Bob's death is terrible, but I need to stay focused. I'll start by gauging the reactions of the other players. Who seems nervous or overly confident?]

Friends, we've lost Bob to this nightmarish creature! We must rely on our wits and cunning to unmask this wolf in sheep's clothing. Let's look for clues in our interactions. Now, who has som

In [ ]:
print(votes)